# HW 2. Свёрточная нейронная сеть

нужно взять train часть от [tiny imagenet](http://cs231n.stanford.edu/tiny-imagenet-200.zip) dataset и обучить ЛЮБУЮ нейросетку

💀 ограничения:  
\- нельзя использовать предобученные модели  
\- максимальное разрешение картинки на входе 128х128  

разбалловка:  
+💎 успешно написан класс dataset для tiny_imagenet. Есть разбиение на две части (одна для обучения, вторая для валидации)  
+💎 использовать аугментации из библиотеки albumentations для увеличения размеров dataset-a  
+💎 успешно реализован алгоритм обучения нейросети  
+💎 accuracy in [45%, 55%)  
+💎 accuracy >= 55%  
Total: 5 / 5 💎  

Оценка accuracy идёт по обученной модели, поэтому в git нужно сохранить веса модели.


In [1]:
import torch
import torch.nn as nn
import torchvision
import timm

import numpy as np
from datetime import datetime
import os
import cv2
import albumentations

import matplotlib.pyplot as plt
%matplotlib inline

## Скачиваем dataset
1. Маунтим Google-Drive к Google-Colab
2. Скачиваем и разархивируем dataset в Google-Drive
3. Чистим временные файлы

In [2]:
DATASET_URL = "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
DATASET_NAME = "tiny-imagenet-200"
SAVE_PATH = "" # Этот путь уже должен быть на google-drive
DATASET_DIR = SAVE_PATH + DATASET_NAME
NUM_CLASSES = 200

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"current device:{device}")

current device:cuda


In [4]:
# Загрузка датасета в google colab
!wget -P "{SAVE_PATH}" "{DATASET_URL}"
!unzip -q "{SAVE_PATH + DATASET_NAME + '.zip'}" -d "{SAVE_PATH}"
!rm -r "{SAVE_PATH + DATASET_NAME + '.zip'}"

--2025-08-20 17:51:18--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.64.64
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cs231n.stanford.edu/tiny-imagenet-200.zip [following]
--2025-08-20 17:51:19--  https://cs231n.stanford.edu/tiny-imagenet-200.zip
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
/content: Permission denied
/content/tiny-imagenet-200.zip: No such file or directory

Cannot write to ‘/content/tiny-imagenet-200.zip’ (Success).
unzip:  cannot find or open /content/tiny-imagenet-200.zip, /content/tiny-imagenet-200.zip.zip or /content/tiny-imagenet-200.zip.ZIP.
rm: cannot remove '/content/tiny-imagenet-200.zip': No such file or directory


In [4]:
# Подключаем Google Drive для резервного копирования файлов модели
# from google.colab import drive
# drive.mount('/content/drive')

DRIVE_PATH = ''
MODEL_FILENAME = "model.pth"
LOG_FILENAME = 'training_log.txt'

In [5]:
import logging
# Настройка логирования состояния в файл. Печатать в консоль = лаги
!rm "{LOG_FILENAME}"
logging.basicConfig(
    filename=LOG_FILENAME,
    level=logging.INFO,
    format='%(asctime)s - %(message)s',
    datefmt='%H:%M:%S',
    force=True,
)

## Готовим dataset

Используемая структура tiny-imagenet-200 dataset:  
- `words.txt` <small>*decoder: object id -> meaning*</small>  
- `wnids.txt` <small>*object id's in curent dataset*</small>   
- `train`   

`train` содержит папки с названием `[object id]` из `wnids.txt`.  
Внутри каждой такой папки лежат:
- папка `images` содержащая картинки `[object_id]_[picture_num].JPEG`
- файл лейблов `[object_id]_boxes.txt` сопоставляющий каждой картинке из `images` некоторый bounding box.

Всего в train dataset 200 классов, ~100000 изображений.  


Начнём с получения списка всех вложенных в датасет классов. Запихнём их в массив и отфильтруем по возрастанию.

In [6]:
class ImgClass():
  def __init__(self, name):
    self.name = name # class id
  def __str__(self):
    return f"{self.name}"

img_classes = np.empty(NUM_CLASSES, dtype=ImgClass)

# img_classes init
with open(os.path.join(DATASET_DIR, "wnids.txt"), "r") as f:
  lines = sorted(f.readlines())
  if len(lines) != NUM_CLASSES: raise SystemError("Not today")
  for idx, line in enumerate(lines):
    class_id = line.strip()
    img_classes[idx] = ImgClass(class_id)

Создадим класс для загрузки данных. Класс будет разбивать tiny-image200 dataset на 2 части с размером зависящим от параметров инициализации. Также при запросе у класса очередного изображения к выдаваемому изображению будет применена аугументация на основе параметров инициализации.

In [7]:
def get_class_img_dir(root_dir:str, img_class:str):
  return os.path.join(root_dir, "train", img_class, "images")

class DatasetCustom(torch.utils.data.Dataset):
  def __init__(self,
    transforms: albumentations.Compose | None,
    train_split=0.9,
    split_type='train',
    root=DATASET_DIR,
    ordered_classes=img_classes,
  ):
    self.files = []
    self.labels = []
    self.transforms = transforms

    for img_class_idx, img_class in enumerate(ordered_classes):
      # получение списка названий всех фотографий класса
      class_img_path = get_class_img_dir(root, str(img_class))
      class_images = sorted(os.listdir(class_img_path))

      # разделение датасета на 2 части: train + valid
      valid_start_idx = int(len(class_images) * train_split)
      if split_type == "train":
        class_images = class_images[:valid_start_idx]
      else:
        class_images = class_images[valid_start_idx:]

      # преобразование списка названий фотографий к абсолютному пути
      class_images = [
        os.path.join(class_img_path, file_) for file_ in class_images
      ]

      # добавление в датасет путей к файлам и лэйблов
      self.files.extend(class_images)
      self.labels.extend([img_class_idx] * len(class_images))

  def __len__(self):
    return len(self.files)

  def __getitem__(self, idx):
    image = cv2.imread(self.files[idx])[:, :, ::-1]

    if self.transforms is not None:
      image = self.transforms(image=image)["image"]
    label = self.labels[idx]

    return image, torch.tensor(label)

Зададим преобразования train + valid изображений в соответсвии с ТЗ:

- При получении очередной картинки из train-dataset к ней будет применяться аугументация. Каждую эпоху одно и тоже train изображение будет подаваться со случайным искажением.  

- Будем делать resize 128x128  


In [69]:
train_transform = albumentations.Compose(
  [
    # случайное исскажение
    albumentations.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
    # тяжёлые операции
    albumentations.GaussianBlur(blur_limit=3, p=0.2), # Размытие
    albumentations.GaussNoise(p=0.2), # Добавление шума
    # resize 128x128
    albumentations.RandomResizedCrop((128,128)),
    albumentations.CoarseDropout(
        num_holes_range=(1,8),        
        hole_height_range=(4,10),        
        hole_width_range=(4,10),
        fill="random",
        p=0.7),
    albumentations.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.9),
    albumentations.HorizontalFlip(p=0.5),
    albumentations.VerticalFlip(p=0.3),
    albumentations.ColorJitter(),
    

    # нормализация
    albumentations.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    albumentations.pytorch.ToTensorV2(),
  ]
)

valid_transform = albumentations.Compose(
  [
    # нормализация
    albumentations.Resize(height=128, width=128),
    albumentations.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    albumentations.pytorch.ToTensorV2(),
  ]
)

/home/mikhail_m/.local/lib/python3.10/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


Разбиваем весь dataset на 2 части: train + valid.


In [53]:
train_dataset = DatasetCustom(train_transform, split_type="train", train_split=0.9)
valid_dataset = DatasetCustom(valid_transform, split_type="valid", train_split=0.9)

In [10]:
len(train_dataset.files), len(valid_dataset.files)

(90000, 10000)

Создаём даталоадеры

In [54]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=256,
    shuffle=True,
    pin_memory=True,
    num_workers=8,
)

valid_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=256,
    shuffle=False,
    pin_memory=True,
    num_workers=8,
)

Будем использовать архитектуру **EfficientNet**

In [68]:
model = timm.create_model(
    'efficientnet_lite0',
    pretrained=False,
    in_chans=3,
    num_classes=0,
    global_pool="avg",
    drop_rate=0.3,
)

in_features = model.num_features
out_features = in_features // 2

model.classifier = nn.Sequential(

    torch.nn.Linear(
      in_features=in_features,
      out_features=out_features,
      bias=False,
    ),
    torch.nn.BatchNorm1d(num_features=out_features),
    torch.nn.ReLU6(inplace=True),

    torch.nn.Linear(
      in_features=out_features,
      out_features=NUM_CLASSES,
    )
)

In [67]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-8)
criterion = torch.nn.CrossEntropyLoss()

In [65]:
# загрузка обученной модели
best_acc = 0.0
MODEL_FILENAME = "model_acc550.pth"
model_filename = os.path.join(DRIVE_PATH, MODEL_FILENAME)

if False:
  checkpoint = torch.load(
      model_filename,
      #MODEL_FILENAME,
      map_location=torch.device(device)
  )
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  best_acc = checkpoint['acc']
  print(f"Loaded model with acc: {best_acc}")

_ = model.to(device), print(f"model loaded to {device}")
_ = model.eval()

model loaded to cuda


Основной цикл обучения

In [16]:
def gpu_used_mb():
    free, total = torch.cuda.mem_get_info(device)
    return (total - free) / 1024 ** 2

In [17]:
from sklearn.metrics import classification_report

In [66]:
N_EPOCH = 100
total_steps = len(train_loader.dataset) // train_loader.batch_size + (1 if len(train_loader.dataset) % train_loader.batch_size != 0 else 0)

# отвечает за отлов переобучения train_acc > val_acc
MAX_TOLERANCE = 4
MAX_TOLERANCE_DELTA = 0.1
tolerance = 0

# отвечает за отлов стагнации val_acc < best_val_acc
MAX_PATIENCE = 4
last_val_acc = 0.0
patience = 0

flag_rolledback = False # флаг отката весов модели на старую версию из-за ошибки
flag_getbettermodel = False # флаг появления модели с лучшей точностью

total, correct = 0, 0
total_loss, train_loss = 0.0, 0.0
train_acc, val_acc = 0.0, 0.0

try:
  # main learn
  logging.info("Learing starting...")
  print("Learing starting...")
  for epoch in range(N_EPOCH):
    if flag_rolledback or flag_getbettermodel:
        patience = 0
        tolerance = 0
        last_val_acc = 0.0
        flag_rolledback = False
        flag_getbettermodel = False
        
    # train
    model.train()

    total, correct, total_loss = 0, 0, 0.0

    for step, (x, y) in enumerate(train_loader):

      optimizer.zero_grad(set_to_none=True)
      x = x.to(device=device, non_blocking=True, dtype=torch.float32)
      y = y.to(device=device, non_blocking=True, dtype=torch.long)

      y_pred = model(x)
      loss = criterion(y_pred, y)

      total_loss += loss.item() * x.size(0)
      pred = y_pred.argmax(1)
      correct += (pred == y).sum().item()
      total += x.size(0)

      loss.backward()
      optimizer.step()
      if step % 25 == 0:
        logging.info(f"{step+1}/{total_steps} | loss={loss.item():.4f} | acc={correct/total:.3f} ")

    # train done
    train_loss = total_loss / total
    train_acc = correct / total

    # valid
    model.eval()

    total, correct = 0, 0
    ys = []
    y_preds = []

    with torch.no_grad():
      for x, y in valid_loader:
        x = x.to(device=device, non_blocking=True, dtype=torch.float32)
        y = y.to(device=device, non_blocking=True, dtype=torch.long)

        y_pred = model(x).argmax(1)
        correct += (y_pred == y).sum().item()
        total += x.size(0)

        ys.extend(y.detach().cpu().tolist())
        y_preds.extend(y_pred.detach().cpu().tolist())

    # valid done
    val_acc = correct / total
    log_str = f"Epoch {epoch+1}/{N_EPOCH} | train_loss={train_loss:.4f} | train_acc={train_acc:.3f} | val_acc={val_acc:.3f} | used GPU={gpu_used_mb()}MB"
    logging.info(log_str), print(log_str)
    logging.info(classification_report(ys, y_preds))

    if val_acc < train_acc and train_acc - val_acc > MAX_TOLERANCE_DELTA:
        tolerance += 1
    else:
        tolerance = 0
    
    if tolerance > MAX_TOLERANCE:
        print(f"Overlearnig: Rolling back to {model_filename}")
        checkpoint = torch.load(
            model_filename,
            map_location=torch.device(device)
        )
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        
        flag_rolledback = True
        continue
        
    # чекпоинт лучшей модели
    if val_acc > best_acc:
        
        model_filename = os.path.join(DRIVE_PATH, f"model_acc{(1000*val_acc):03.0f}.pth")
        if not os.path.exists(model_filename):
            best_acc = val_acc
            checkpoint = {
              'acc': val_acc,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
            }
            torch.save(checkpoint, model_filename)
            
            flag_getbettermodel = True
            continue
    else:
        if last_val_acc < val_acc :
            # если всё ещё есть прогресс в обучении
            last_val_acc = val_acc
            continue
        
        patience += 1
        if patience > MAX_PATIENCE:
            print(f"no raise val_acc: Rolling back to {model_filename}")
            checkpoint = torch.load(
                model_filename,
                map_location=torch.device(device)
            )
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            
            flag_rolledback = True
            continue

finally:
  # резервное копирование файла логов
  time = datetime.now().strftime('%H%M%S')
  # файл логирования
   #drive_log_name = f"log_colab_{ time }.txt"
   #drive_log_path = os.path.join(DRIVE_PATH , drive_log_name)
   #!cp "{LOG_FILENAME}" "{drive_log_path}"
  # текущая модель
  drive_log_name = f"curent_model_{ time }.pth"
  drive_log_path = os.path.join(DRIVE_PATH , drive_log_name)
  checkpoint = {
    'acc': correct / total,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
  }
  torch.save(checkpoint, drive_log_path)


Learing starting...
Epoch 1/100 | train_loss=5.3753 | train_acc=0.005 | val_acc=0.007 | used GPU=13511.0MB


/home/mikhail_m/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/mikhail_m/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/mikhail_m/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Epoch 2/100 | train_loss=5.3764 | train_acc=0.005 | val_acc=0.006 | used GPU=13511.0MB


/home/mikhail_m/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/mikhail_m/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/mikhail_m/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Epoch 3/100 | train_loss=5.3777 | train_acc=0.005 | val_acc=0.006 | used GPU=13511.0MB


/home/mikhail_m/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/mikhail_m/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/mikhail_m/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Epoch 4/100 | train_loss=5.3763 | train_acc=0.005 | val_acc=0.004 | used GPU=13511.0MB


/home/mikhail_m/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/mikhail_m/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/mikhail_m/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


KeyboardInterrupt: 

Жёсткая полная очистка GPU памяти:  
`nvidia-smi` узнаём PID python процесса  
`kill <PID>`  

In [62]:
import gc
print(gpu_used_mb())
torch.cuda.empty_cache()
gc.collect()
print(gpu_used_mb())

7501.0
7501.0
